# Finding the closes household with Fiber

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from parsers import get_closest_fiber
from aggregators import filter_df

In [5]:
# inputs
fn_att = '../data/output/speed_price_att.csv.gz'
fn_centurylink = '../data/output/speed_price_centurylink.csv.gz'
fn_verizon = '../data/output/speed_price_verizon.csv.gz'
fn_earthlink = '../data/output/speed_price_earthlink.csv.gz'
inputs = {
    "AT&T" : fn_att,
    "CenturyLink": fn_centurylink,
    "Verizon": fn_verizon,
    "EarthLink" : fn_earthlink
}

In [13]:
??get_closest_fiber

Signature: get_closest_fiber(df: pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame
Source:   
def get_closest_fiber(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert coordinates to radians and fit a sklearn ball tree 
    to find closest household with 200 Mbps speeds.
    """
    _df = df[df.speed_down >= 200]
    
    # create a ball tree just on fiber households
    tree = BallTree(np.deg2rad(_df[['lat', 'lon']]), metric="haversine")
    # find the closest fiber for every household
    distances, indices = tree.query(np.deg2rad(df[['lat', 'lon']]),
                                    k=1, return_distance=True)
    df["closest_fiber_miles"] = distances * 3958.756
    
    # merge the info of the closest fiber household
    closest = _df.iloc[indices[:, 0]].reset_index(drop=True) 
    return df.merge(closest, 
                    how='left',
                    left_index=True, right_index=True, 
                    suffixes=['', '_closest_fiber'])
File:      ~/code/1_p

In [7]:
view_cols = [
    'address_full',
    'address_full_closest_fiber',
    'closest_fiber_miles',
    'speed_down',
    'median_household_income',
    'median_household_income_closest_fiber',
    'race_perc_non_white',
    'race_perc_non_white_closest_fiber'
    
]

In [8]:
for isp, fn in inputs.items():
    df = filter_df(fn, isp)
    break

In [14]:
for city, df__ in df.groupby('major_city'):
    df__ = get_closest_fiber(df__.reset_index(drop=True))
    print(df__[df__.speed_down < 200][view_cols].sort_values('closest_fiber_miles').head(3))
    break

                          address_full        address_full_closest_fiber  \
9625    30 5th St NE, Atlanta GA 30308   402 5th St NE, Atlanta GA 30308   
5760  199 14th St NE, Atlanta GA 30309  194 14th St NW, Atlanta GA 30318   
5748  181 14th St NE, Atlanta GA 30309  194 14th St NW, Atlanta GA 30318   

      closest_fiber_miles  speed_down  median_household_income  \
9625                  0.0        18.0                 112778.0   
5760                  0.0        50.0                 109674.0   
5748                  0.0        18.0                 109674.0   

      median_household_income_closest_fiber  race_perc_non_white  \
9625                               112778.0             0.233680   
5760                               109674.0             0.251443   
5748                               109674.0             0.251443   

      race_perc_non_white_closest_fiber  
9625                           0.233680  
5760                           0.251443  
5748                           

In [22]:
# get the three closest address pairs where an address in a low income area is closest to a blazing fast offer to an upper income area.
df__[(df__.income_level == 'Low') & 
     (df__.income_level_closest_fiber == 'Upper Income')].sort_values('closest_fiber_miles')[view_cols].head(3)

,address_full,address_full_closest_fiber,closest_fiber_miles,speed_down,median_household_income,median_household_income_closest_fiber,race_perc_non_white,race_perc_non_white_closest_fiber
9605,"729 Brittain Dr NW, Atlanta GA 30313","708 Spring St NW, Atlanta GA 30308",0.210075,10.0,31042.0,115500.0,0.576454,0.266111
1260,"1664 Briarcliff Rd NE, Atlanta GA 30306","1776 Johnson Rd NE, Atlanta GA 30306",0.210195,50.0,30486.0,170096.0,0.533031,0.140210
1274,"120 North Ave NW, Atlanta GA 30313","699 Spring St NW, Atlanta GA 30308",0.221963,5.0,23250.0,115500.0,0.588896,0.266111
